In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import scipy.stats
from IPython.display import Video, display, HTML
import re
from colour import Color

sns.set()


def gethighest(vals, prefix="logall_"):
        m = np.argmax([int(re.search(f'{prefix}(\d+)', val).group(1)) for val in vals])
        return vals[m]

def read_csvx(path, *args, **kwargs):
    try:
        return pd.read_csv(str(path) +'.gz', *args, **kwargs)
    except FileNotFoundError as e:
        return pd.read_csv(str(path), *args, **kwargs)

        
def readparams(file):
    params = {} 
    with open(file) as f:
        for line in f:
            line = line[:-1]  # remove \n
            if not line or line.startswith('#') or line.startswith('import'):
                continue
            key, val = line.split('=')
            if val.lower() == 'true' or val.lower() == 'false':
                params[key] = val.lower() == 'true'
            else:
                try:
                    params[key] = float(val)
                except ValueError:
                    params[key] = val
    return params

In [ ]:
from IPython.display import HTML, display
from ipywidgets import widgets
a = widgets.Text()
import uuid
classname='user-agent-content'+str(uuid.uuid4())
a.add_class(classname)
display(a)

In [ ]:
HTML("""
<script>
'use strict';
// Get the element
var inp = document.getElementsByClassName(
     '""" + str(classname) + """')[0].getElementsByTagName('input')[0];

// Set the UA
inp.value = window.navigator.userAgent;

// Trigger change for Jupyter lab kernel
var evt = new Event('change', {"bubbles":true, "cancelable":false});
inp.dispatchEvent(evt);
</script>
User Agent set
""")

In [ ]:
if 'Mac OS' in a.value :
    %config InlineBackend.figure_format = 'retina'
    print('retina set')


In [ ]:
dfful = pd.read_pickle('all_file_loaded_lionscross10-procbtest.pkl.gz')
dfcond = dfful.query('(gNbOfPhysicalObjects == 10 or gNbOfPhysicalObjects == 20 or gNbOfPhysicalObjects == 40) and (nOpti == 2 or nOpti == 4)')

In [ ]:
%matplotlib inline
sns.set_context('paper')
sns.set_style('white')

## Colors
coopColor = Color("#3375b2")
defectColor = Color("#963d35")
badtoomany = Color("#963d35")
badtoomany.luminance *= 2
badnotenough = Color("#963d35")
badnotenough.luminance *= 1
good = Color("#3375b2")


## Helpers
def plot_ess(data, *args, **kwargs):
    nopti = data['nOpti'].mean()
    plt.axhline(5/nopti, *args, **kwargs)
    
def plot_so(data, *args, **kwargs):
    nopti = data['nOpti'].mean()
    plt.axhline(5 + 10 * (nopti-1)/nopti, *args, **kwargs)
def plot_span(data, *args, **kwargs):

    nhat = data['nOpti'].mean()
    omega = data['gNbOfPhysicalObjects'].mean()
    vs = plt.axvspan(0, nhat+1, alpha=0.1, color=badnotenough.hex, label="Not enough agents")
    vs.set_zorder(0)
    vs = plt.axvspan(nhat+1, min(nhat*omega+1, 102), alpha=0.1, color=good.hex, label="Enough opportunities and agents")
    vs.set_zorder(0)
    vs = plt.axvspan(min(nhat*omega+1, 102), 102, alpha=0.1, color=badtoomany.hex, label="Not enough opportunities")
    vs.set_zorder(0)

    
def annotator(itera, *args, size=15, **kwargs):
    plt.text(-0.05, 1.05, next(itera) + '.', transform=plt.gca().transAxes, 
            size=size, weight='bold')


In [ ]:
import string
g = sns.relplot(data=dfcond.query('nOpti == 2'), col="gNbOfPhysicalObjects", x="maxPlayer", y='truecoopmod' , kind="line",
                facet_kws={'ylim':(-0.5, 12.5), 'xlim':(0, 102), 'sharey': True, 'legend_out': True}, aspect=1.2, height=2.5)
g.map_dataframe(sns.scatterplot, x="maxPlayer", y="truecoopopti")
#g.map_dataframe(sns.scatterplot, x="maxPlayer", y="truecoopmod", color="green")

g.map_dataframe(plot_ess,  color=defectColor.hex, label="defect investment")
g.map_dataframe(plot_so,  color=coopColor.hex, label="social optimum investment")
g.map_dataframe(plot_span)
g.set_titles("$\omega = {col_name}$")
g.set(ylabel='Mean investment', xlabel="Number of agents in the environment")
g.map(annotator, itera=iter(string.ascii_lowercase))

#g.set_xlabel('Number of agents in the environment')
g.add_legend()
g.savefig('out/varNrowOpp.pdf')
plt.show()

In [ ]:
g = sns.relplot(data=dfcond, col="gNbOfPhysicalObjects", row="nOpti", x="maxPlayer", y='truecoopmod' , kind="line",
                facet_kws={'ylim':(-0.5, 12.5), 'xlim': (0, 102)}, aspect=1.2, height=2.5)

#g.map_dataframe(sns.scatterplot, color="green", x="maxPlayer", y="truecoopmod")
g.map_dataframe(sns.scatterplot, x="maxPlayer", y="truecoopopti")
g.map_dataframe(plot_ess, color=defectColor.hex, label="Defect ESS")
g.map_dataframe(plot_so, color=coopColor.hex, label="Social optimum ESS")
g.map_dataframe(plot_span)
g.map(annotator, itera=iter(string.ascii_lowercase))

g.set_titles('$\hat{{n}} = {row_name}$ | $\omega = {col_name}$')
g.set(ylabel='Mean investment', xlabel="Number of agents in the environment")
g.add_legend()
g.savefig('out/grid.pdf')


In [ ]:
dfful.head()

In [ ]:
# AJOUTER EXPÉRIENCE CONTRÔLE SANS VISION

In [ ]:
plt.figure(figsize=(5, 3))
dfobjvar=dfful.query('maxPlayer == 100 and nOpti == 2')
plt.axvspan(0, 50, color=badtoomany.hex, alpha=0.1, label="Not enough opportunities")
plt.axvspan(50, 102, color=good.hex, alpha=0.1, label="Enough opportunities")
sns.lineplot(data=dfobjvar, x='gNbOfPhysicalObjects', y='truecoopopti')
sns.scatterplot(data=dfobjvar, x='gNbOfPhysicalObjects', y='truecoopopti')
sns.despine()
plt.ylim(-0.1, 14)
plt.xlim(0, 102)
plot_so(dfobjvar, color=coopColor.hex, label="Social optimal investment")
plot_ess(dfobjvar, color=defectColor.hex, label="Defector investment")
plt.legend()
plt.xlabel('Number of cooperation opportunities $\omega$')
plt.ylabel('Mean investment for $\hat{n}$')
plt.tight_layout()
plt.savefig('out/varopp.pdf')